Introduction

This notebook will introduce some base API functionality of the ESMFold LLM, which is a Metagenomics Atlas. Allowing us to Fold a protein sequence to see the resultant predicted structure.

We will use the following Python libraries:

- PyTorch
- esm
- prothelpers
- py3Dmol

Will walkthrough each stage step-by-step carrying out the following taks:

1. Download Protein PDB data
2. Parse the protein data
3. Visualise protein data
4. Extract an experimental structure and sequence of chain
5. Setup a pre-trained ESMFold model using AutoTokenizer from HuggingFace
6. Tokenise sequence (experimental structure from step 4.) to convert it into numerical format (so can be used by ESMFold for prediciton)
7. Use instantiated ESMFold Model to make prediction
8. Submit tokenised sequence to ESMFold model to make prediction of the 3D Structure (May take ~5 minutes on non hardware accelerated setup)
9. View result visualisation
10. Evaluate accuracy of prediction by comapring to experimental structure.
11. Calculate TM-Score and RMSD to evaluate overall prediction experimentation
12. Finish
